# Text Document Classification with KNN and Naive Bayes

You will classify text documents using Naive Bayes classifers.  We will be working with the dataset called "20 Newsgroups", which is a collection of 20,000 newsgroup posts organized into 20 categories.

## 1. Loading the 20 Newsgroups Dataset
The dataset is called “20 Newsgroups”. Here is the official description, quoted from the [website](http://qwone.com/~jason/20Newsgroups/)
>The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge, it was originally collected by Ken Lang, probably for his paper “Newsweeder: Learning to filter netnews,” though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

In [1]:
#First we need to initialize Python.  Run the below cell.
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from sklearn.pipeline import Pipeline
import operator

from numpy import *
from sklearn import *
from scipy import stats
random.seed(4487)

- Put the file "20news-bydate_py3.pkz' into the same directory as this ipynb file. **Do not unzip the file**.

- Extract 4 classes ('alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space') from the dataset. 

In [2]:
# strip away headers/footers/quotes from the text
removeset = ('headers', 'footers', 'quotes')

# only use 4 categories
cats      = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# load the training and testing sets
newsgroups_train = datasets.fetch_20newsgroups(subset='train',
                           remove=removeset, categories=cats, data_home='./')
newsgroups_test  = datasets.fetch_20newsgroups(subset='test', 
                           remove=removeset, categories=cats, data_home='./')

- Check if we got all the data.  The training set should have 2034 documents, and the test set should have 1353 documents.

In [3]:
print("training set size:", len(newsgroups_train.data))
print("testing set size: ",  len(newsgroups_test.data))
print(newsgroups_train.target_names)

training set size: 2034
testing set size:  1353
['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


- Count the number examples in each class.  `newsgroups_train.target` is an array of class values (0 through 3), and `newsgroups_train.target[i]` is the class of the i-th document.

In [4]:
print("class counts")
for i in [0, 1, 2, 3]:
    print("{:20s}: {}".format(newsgroups_train.target_names[i], sum(newsgroups_train.target == i)))

class counts
alt.atheism         : 480
comp.graphics       : 584
sci.space           : 593
talk.religion.misc  : 377


- Show the documents.  `newsgroups_train.data` is a list of strings, and `newsgroups_train.data[i]` is the i-th document.

In [5]:
print("--- document {} (class={}) ---".format(
    0, newsgroups_train.target_names[newsgroups_train.target[0]]))
print(newsgroups_train.data[0])

--- document 0 (class=comp.graphics) ---
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych


**Tip:** while you do the tutorial, it is okay to make additional code cells in the file.  This will allow you to avoid re-running code (like training a classifier, then testing a classifier).

## 2. Extracting Features from Text Files
In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors.

Next, we will introduce two basic text representation methods: One-hot encoding, Bag of words, and TF-IDF. More feature vector extraction functions, please refer to https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction

### one-hot encoding
- Each word is coded with an index, which is represented by one-hot.

> John likes to watch movies. Mary likes too.

> John also likes to watch football games.

If we need to represent the words in the above two sentences, you can encode the words as following:

> {"John": 1, "likes": 2, "to": 3, "watch": 4, "movies": 5, "also":6, "football": 7, "games": 8, "Mary": 9, "too": 10}

We can encode each word using one-hot method

>John: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

>likes: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

>...

#### However, this text representation method is impractical when the scale of corpus becomes large.

### Bag of Words
- The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

> John likes to watch movies. Mary likes too.  -->> [1, 2, 1, 1, 1, 0, 0, 0, 1, 1]

> John also likes to watch football games.     -->> [1, 1, 1, 1, 0, 1, 1, 1, 0, 0]

The **sklearn.feature_extraction.text.CountVectorizer** implement the `Bag of Words` method that converts a collection of text documents to a matrix of token counts. This implementation produces a sparse representation of the counts using **scipy.sparse.coo_matrix** to save memory by only storing the non-zero parts of the feature vectors in memory.

### Term Frequency - Inverse Document Frequency (TF-IDF)

In the word bag model, we can get the vector representation of this text.However, in the face of the diversity of text, each word has different weight to the content of text in practical application, so we introduce tf-idf model.

##### TF (Term Frequency)

In the case of the term frequency $tf(t, d)$, the simplest choice is to use the raw count of a term in a document, i.e., the number of times that term $t$ occurs in document $d$. If we denote the raw count by $f_{t, d}$, then the simplest tf scheme is $tf(t,d) = f_{t, d}$. 

$tf_{t, d} = n_{t, d}/\sum_kn_{t, d}$ (divided)

The numerator in the above formula is the number of occurrences of the word in the document $d$, and the denominator is the sum of the occurrences of all words in the document $d$.

##### IDF (Inverse Document Frequency) 

The inverse document frequency is a measure of how much information the word provides, i.e., if it's common or rare across all documents. It is the logarithmically scaled inverse fraction of the documents that contain the word (obtained by dividing the total number of documents by the number of documents containing the term, and then taking the logarithm of that quotient): 

$idf(t ,D) = log\frac{N}{|\{ d\in D:t \in d \}|}$

with 
- $N$: total number of documents in the corpus $N=|D|$
- $|\{ d\in D:t \in d \}|$: number of documents where the term $t$ appears. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to  $1+|\{ d\in D:t \in d \}|$

Then tf-idf is calculated as: 
$tfidf(t, d, D) = tf(t, d) * idf(t, D)$

Both tf and tf–idf can be computed as follows using **sklearn.feature_extraction.text.TfidfTransformer**.

In [6]:
# This is Token count (Simple bag of words)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
 
corpus_example = ['This is the first document.',
	'This document is the second document.',
	'And this is the third one.',
	'Is this the first document?']
 
pipe_example = Pipeline([('bag_of_words', CountVectorizer()),
                 ('tfid', TfidfTransformer())]).fit(corpus_example)
print(pipe_example['bag_of_words'].transform(corpus_example).toarray())
print(pipe_example['tfid'].idf_)
results_tfid_example = pipe_example.transform(corpus_example)
print(pipe_example['bag_of_words'].get_feature_names())
print(results_tfid_example.toarray())

[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


Create the vocabulary from the training data.  Then use **sklearn.feature_extraction.text.CountVectorizer** to build the document vectors for the training and testing sets.  You can decide how many words you want in the vocabulary

In [7]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(newsgroups_train['data'])
print(X_train.toarray().shape)


(2034, 26576)


In [8]:
sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(0), reverse = True)[100:150]

[('ysc', 26475),
 ('yr', 26474),
 ('yoyodyne', 26473),
 ('youve', 26472),
 ('youth', 26471),
 ('yousung', 26470),
 ('yourselfers', 26469),
 ('yourdon', 26468),
 ('youngsters', 26467),
 ('youngster', 26466),
 ('young', 26465),
 ('yoshiro', 26464),
 ('yorku', 26463),
 ('york', 26462),
 ('yorg', 26461),
 ('yorba', 26460),
 ('yor', 26459),
 ('yoke', 26458),
 ('yo', 26457),
 ('yngvesson', 26456),
 ('yktvmh', 26455),
 ('yingyong', 26454),
 ('yields', 26453),
 ('yielding', 26452),
 ('yielded', 26451),
 ('yield', 26450),
 ('yhwh', 26449),
 ('yhvh', 26448),
 ('yfn', 26447),
 ('yf', 26446),
 ('yesterday', 26445),
 ('yes', 26444),
 ('yer', 26443),
 ('yep', 26442),
 ('yeltsin', 26441),
 ('yellows', 26440),
 ('yellow', 26439),
 ('yelling', 26438),
 ('yell', 26437),
 ('yee', 26436),
 ('yeasteryears', 26435),
 ('yeast', 26434),
 ('years', 26433),
 ('yearly', 26432),
 ('year', 26431),
 ('yeah', 26430),
 ('yeager', 26429),
 ('yea', 26428),
 ('ye', 26427),
 ('ydobyns', 26426)]

In [9]:
sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(0))[100:150]

[('0674', 100),
 ('068', 101),
 ('0695', 102),
 ('07', 103),
 ('070', 104),
 ('071', 105),
 ('0729', 106),
 ('0739', 107),
 ('074', 108),
 ('07410', 109),
 ('07653', 110),
 ('077', 111),
 ('08', 112),
 ('08080', 113),
 ('0820', 114),
 ('08540', 115),
 ('08544', 116),
 ('0856e16', 117),
 ('0865', 118),
 ('08786', 119),
 ('08934', 120),
 ('09', 121),
 ('0900', 122),
 ('0901', 123),
 ('0903', 124),
 ('0908', 125),
 ('0926', 126),
 ('0941', 127),
 ('0943', 128),
 ('0970', 129),
 ('0971', 130),
 ('0988', 131),
 ('0_', 132),
 ('0a', 133),
 ('0b', 134),
 ('0e9', 135),
 ('0km', 136),
 ('0mph', 137),
 ('0w', 138),
 ('0x', 139),
 ('0x00', 140),
 ('0x100', 141),
 ('0x1f', 142),
 ('0x3d4', 143),
 ('0xc010', 144),
 ('0xc018', 145),
 ('10', 146),
 ('100', 147),
 ('1000', 148),
 ('10000', 149)]

In [10]:
# Delete words that doesnt have a count token higher than 200
# I did this trying to eliminate outliers and unsignificant words such as '00', '0000' or '3pm'.
# Also words that are repeated in every text doesnt have many importance for us, therefore we want to delete the most common words
# 1. 18.000 > x > 1000
new_vocabulary = dict(filter(lambda dict_elements: 20000 > dict_elements[1] > 2000,vectorizer.vocabulary_.items()))

In [11]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words='english', vocabulary = new_vocabulary.keys())
X_train = vectorizer.fit_transform(newsgroups_train['data'])

## 3. K Nearest Neighbor (KNN)
Let's train a K Nearest Neighbor (KNN) model. Using cross-validation to select the best K parameter. Then, showing the accuracy of training and testing set.

In [12]:

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier as knn

Y_train = newsgroups_train['target']

paramgrid = {'n_neighbors': [3,4,5,7,10],
            'weights': ['distance']}
grid_search = GridSearchCV(knn(), param_grid=paramgrid, cv=10, n_jobs=-1, scoring='accuracy')

    
print(X_train.shape)
    
grid_search.fit(X_train.toarray(), Y_train)
print('Accuracy: ' , grid_search.best_score_)
print('Params: ', grid_search.best_params_)
    


(2034, 17999)
Accuracy:  0.471030136192408
Params:  {'n_neighbors': 3, 'weights': 'distance'}


Accuracy of train set = 0.471

In [ ]:
from sklearn.metrics import accuracy_score

Y_test = newsgroups_test['target']
X_test = vectorizer.transform(newsgroups_test['data']).toarray()

Y_pred = grid_search.predict(X_test)
accuracy_score(Y_test, Y_pred)

Accuracy of test set = 0.433

## 4. Bernoulli Naive Bayes 
Learn a Bernoulli Naive Bayes model from the training set.  What is the prediction accuracy on the test set?  Try different parameters (alpha, max_features, etc) to get the best performance.

In [ ]:
from sklearn.naive_bayes import BernoulliNB

paramgrid = {'alpha' : [.01, .05, 0.1, 0.2]}
grid_search = GridSearchCV(BernoulliNB(), param_grid=paramgrid, cv=10, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train.toarray(), Y_train)

In [ ]:
print(f'Prediction accuracy test set: {grid_search.best_score_}')

In [ ]:
accuracy_score(grid_search.predict(X_test), Y_test)

In [ ]:
grid_search.best_params_ 

What are the most informative words for each category?  Run the below code.

Note: `model.coef_[i]` will index the scores for the i-th class

In [ ]:
bmodel = BernoulliNB(alpha = 0.01)
bmodel.fit(X_train.toarray(), Y_train)

In [ ]:
fnames = asarray(vectorizer.get_feature_names())
for i,c in enumerate(newsgroups_train.target_names):
    tmp = argsort(bmodel.coef_[i])[-10:]
    print("class", c)
    for t in tmp:
        print("    {:9s} ({:.5f})".format(fnames[t], bmodel.coef_[i][t]))

## 5. Multinomial Naive Bayes model
Now learn a multinomial Naive Bayes model using the TF-IDF representation for the documents.  Again try different parameter values to improve the test accuracy.

In [ ]:
### INSERT YOUR CODE HERE
## HINT 
# 1. feature_extraction.text.TfidfTransformer(use_idf=True, norm= )
# 2. naive_bayes.MultinomialNB(alpha= )
from sklearn.naive_bayes import MultinomialNB

pipe_tfidf = Pipeline([('bag_of_words', CountVectorizer(vocabulary  = new_vocabulary.keys())),
                     ('tf-idf', TfidfTransformer())]).fit(newsgroups_train['data'])

X_train = pipe_tfidf.transform(newsgroups_train['data']).toarray()
X_test = pipe_tfidf.transform(newsgroups_test['data']).toarray()
print(X_train.shape)


In [ ]:
paramgrid = {'alpha' : [.01, .05, 0.1, 0.2],
            'fit_prior': [1, 0]}
grid_search = GridSearchCV(MultinomialNB(), param_grid=paramgrid, cv=10, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, Y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
model_MNB = MultinomialNB(alpha=grid_search.best_params_['alpha'], 
                          fit_prior = grid_search.best_params_['fit_prior']
                         ).fit(X_train, newsgroups_train.target)
Y_predict = model_MNB.predict(X_test)
print(accuracy_score(newsgroups_test.target, Y_predict))

What are the most informative features for Multinomial model? Run the below code.

In [ ]:
# get the word names
fnames = asarray(pipe_tfidf['bag_of_words'].get_feature_names())
for i,c in enumerate(newsgroups_train.target_names):
    tmp = argsort(model_MNB.coef_[i])[-10:]
    print("class", c)
    for t in tmp:
        print("    {:9s} ({:.5f})".format(fnames[t], model_MNB.coef_[i][t]))

How do the most informative words differ between the TF-IDF multinomial model and the Bernoulli model?

There is a bigg difference in the most informative words between the two models.
Looking at the class labels and most improtant words of each one, We can say that the Bernulli model's words represent more the class than the words obtained from TF-IDF. That could be seen in the level of importance of each word: in Bernulli it is between -1.x and -2.x in most cases and in Multinominal its -5.x or -4.x.
Also, in Bernulli model we can see words that make more sense with the class label where the text belongs. For example:
1. alt.atheism: atheist, islam, atheism, god (in both models), and believe (in both models).
2. comp.graphics: file, files, image, graphics (all last four in both models), and format.
3. sci.space: earth, lunar, launch, data.
4. talk.religion.misc: christians, bible, believe (in both models), jesus, god (in both models).



## 6. Effect of smoothing
The smoothing (regularization) parameter has a big effect on the performance.  Using the Multinomial TF-IDF models, make a plot of accuracy versus different values of alpha. For each alpha, you need to train a new model. Which alpha value yields the best result?

In [ ]:
### INSERT YOUR CODE HERE
## HINT
# 1. Iterating: alphas = logspace(-5,0,50)
import numpy as np

model_accuracy_alpha = []
for alpha_param in logspace(-10,0,30):
    
    model_MNB = MultinomialNB(alpha= alpha_param).fit(X_train, newsgroups_train.target)
    Y_predict = model_MNB.predict(X_test)
    curr_accuracy = accuracy_score(newsgroups_test.target, Y_predict)
    model_accuracy_alpha.append([curr_accuracy, alpha_param])
      
model_accuracy_alpha = np.array(model_accuracy_alpha)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_accuracy_alpha[:,1], model_accuracy_alpha[:,0])
plt.xlabel('Alpha Param')
plt.ylabel('Model accuracy')
plt.title('Model accuracy vs Alpha smoothing value')
plt.show()

## 7. Effect of vocabulary size
The vocabulary size also affects the accuracy.  Make another plot of accuracy versus vocabulary size.  Which vocabulary size yields the best result?

In [ ]:
model_accuracy_vocabulary = []

for vocabulary_size in linspace(100,26577,20):
    vectorizer = TfidfVectorizer(max_features = int(vocabulary_size))
    
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    X_test = vectorizer.transform(newsgroups_test.data)
    
    model_MNB = MultinomialNB(alpha= 0.01).fit(X_train, newsgroups_train.target)
    Y_predict = model_MNB.predict(X_test)
    curr_accuracy = accuracy_score(newsgroups_test.target, Y_predict)
    model_accuracy_vocabulary.append([curr_accuracy, vocabulary_size])
      

model_accuracy_vocabulary = np.array(model_accuracy_vocabulary)

In [ ]:
plt.plot(model_accuracy_vocabulary[:,1], model_accuracy_vocabulary[:,0])
plt.xlabel('Vocabulary Size')
plt.ylabel('Model Accuracy')
plt.title('Model accuracy vs Vocabulary Size')
plt.show()

The best resoult should be a vocabulary size of 12.500
